<a href="https://colab.research.google.com/github/Temmybjt/Store-Sales--Time-Series-Forecasting/blob/main/Store_Sales_Time_Series_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Store Sales - Time Series Forecasting**

Build a model that more accurately predicts the unit sales for thousands of items sold at different Favorita stores with an approachable training dataset of dates, store, and item information, promotions, and unit sales.

In [ ]:
# Libraries
import pandas as pd; pd.set_option('display.max_columns', 30)
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt; plt.style.use('ggplot')

import plotly.express as px

import warnings; warnings.filterwarnings('ignore')

# Import the main analysis datasets

In [ ]:
# Check the training and test dataframes
train = pd.read_csv('/home/Store Sales/train.csv.zip')
test = pd.read_csv('/home/Store Sales/test.csv')

print('The dimension of the train dataset is:', train.shape)
print('The dimension of the test dataset is:', test.shape)

In [ ]:
train.head(2)

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
test.describe()

In [ ]:
test.info()

# Import supporting datasets

In [ ]:
holidays = pd.read_csv('/home/Store Sales/holidays_events.csv')
stores = pd.read_csv('/home/Store Sales/stores.csv')
transactions = pd.read_csv('/home/Store Sales/transactions.csv.zip')
oil = pd.read_csv('/home/Store Sales/oil.csv')

In [ ]:
holidays.describe()

In [ ]:
stores.describe()

In [60]:
transactions.describe()

,store_nbr,transactions
count,83488.000000,83488.000000
mean,26.939237,1694.602158
std,15.608204,963.286644
min,1.000000,5.000000
25%,13.000000,1046.000000
50%,27.000000,1393.000000
75%,40.000000,2079.000000
max,54.000000,8359.000000


In [61]:
oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [62]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


# Merge datasets with common feature

In [63]:
# Merge oil price
train = train.merge(oil, on='date', how='left')
test = test.merge(oil, on='date', how='left')

# Merge transactions
train = train.merge(transactions, on=['date', 'store_nbr'], how='left')
test = test.merge(transactions, on=['date', 'store_nbr'], how='left')

# Merge stores description
train = train.merge(stores, on='store_nbr', how='left')
test = test.merge(stores, on='store_nbr', how='left')


In [74]:
# Merge holidays after removing transferred holidays
# Remove transferred holidays
holidays.head()

0    False
1    False
2    False
3    False
4    False
Name: transferred, dtype: bool

In [69]:
# Add holidays
train = train.merge(holidays, on='date', how='left')
test = test.merge(holidays, on='date', how='left')

KeyError: 'date'

In [66]:
train.head(2)

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,transactions,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False


In [67]:
train.isna().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
dcoilwtico       955152
transactions     249117
city                  0
state                 0
type_x                0
cluster               0
type_y          2551824
locale          2551824
locale_name     2551824
description     2551824
transferred     2551824
dtype: int64

# Data Exploration